## Using pretrained VGG model for transfer learning

In this example I will be using Keras with tensorflow backend. The data used is taken from cats and dogs classification problem hosted on Kaggle, but later been preprocessed for fast.ai course.  You can find the data [here](http://files.fast.ai/files/dogscats.zip).

In [1]:
## Importing required libraries
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Input,Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam
%matplotlib inline
## Path of your data folder
path = "C:/Users/Aayush - Carlson/Downloads/fastai/deeplearning1/data/dogscats/"

Using TensorFlow backend.


In [2]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=True)

## Removing the last layer of the model
model_vgg16_conv.layers.pop()
model_vgg16_conv.outputs = [model_vgg16_conv.layers[-1].output]
model_vgg16_conv.output_layers = [model_vgg16_conv.layers[-1]] # added this line in addition to zo7 solution
model_vgg16_conv.layers[-1].outbound_nodes = []

## Setting the model layers to be non trainable
for layer in model_vgg16_conv.layers: layer.trainable=False
    
## Adding new layer to the model
input = Input(shape=(224,224,3),name = 'image_input')
output_vgg16_conv = model_vgg16_conv(input)
x = Dense(2, activation='softmax', name='predictions')(output_vgg16_conv)

#Adding new layer to the model
my_model = Model(input=input, output=x)
model_vgg16_conv.summary()
my_model.summary()
#my_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
my_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

C:\Users\Aayush - Carlson\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("pr..., inputs=Tensor("im...)`


In [3]:
## Using same preprocessing done on original VGG dataset, which is subtracting mean of original imagenet images from new images
def preprocess_input(x):
    x = x[:,:,::-1]
    x[:,:,0] -= 103.939
    x[:,:,1] -= 116.779
    x[:,:,2] -= 123.68
    return x[:,:,::-1]

## Function for fetching data in batches
def get_batches(path, gen=ImageDataGenerator(preprocessing_function = preprocess_input), 
                shuffle=True, batch_size=8, class_mode='categorical'):
    return gen.flow_from_directory(path,target_size=(224,224),class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

"\ndef get_batches(path, gen=ImageDataGenerator(), \n                shuffle=True, batch_size=8, class_mode='categorical'):\n    return gen.flow_from_directory(path,target_size=(224,224),class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)"

In [5]:
## Getting data from the train and validation folder
batch_size=8 ## Set this number as high as your gpu memory supports - Mine is a GTX 950M - 2 GB graphic card)
batches = get_batches(path+'train', batch_size=batch_size) # Getting training images
val_batches = get_batches(path+'valid', batch_size=batch_size*2) ## Getting validation images

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
## Fitting the model
my_model.fit_generator(batches, steps_per_epoch= batches.n ,validation_data = val_batches,validation_steps = val_batches.n, epochs=1)

Epoch 1/1
 5922/23000 [======>.......................] - ETA: 4991s - loss: 0.1523 - acc: 0.9774

### As we can see just in one epoch we were able to achieve >98% accuracy which would have won the Kaggle competition at the time it was launched. Transfer learning is a very powerful technique and should always be used unless the dataset is totally different from imagenet.